In [6]:
import pandas as pd
import numpy as np
!pip install -U -q PyDrive
from pydrive.drive import GoogleDrive
from google.colab import drive

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense

In [7]:
# # data_path = '/home/arsh/Jasleen/Spring 2023/NLP/Group Project/Authorship-Attribution-for-Neural-Text-Generation-master/data/'
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data_path = 'drive/MyDrive/NLP/'

In [5]:
data = pd.read_csv(data_path+'input.csv')
# data.tail()

In [9]:
# Creating new pairs

import pandas as pd
import numpy as np

# Load the data from the CSV file
# df = pd.read_csv('input.csv')
df=data

# Select only 100 samples from each class
grouped = df.groupby('class').apply(lambda x: x.sample(n=min(len(x), 1000))).reset_index(drop=True)

# Group the selected data by the "class" column
grouped = grouped.groupby('class')

# Randomly select 10 rows from each group
samples = []
for _, group in grouped:
    samples.extend(list(zip(group.sample(n=1000, replace=True).index, 
                             group.sample(n=1000, replace=True).index)))


# Generate pairs from the selected rows and random rows from other classes
pairs = []
labels = []
for i, j in samples:
    # Positive pair
    if df['class'][i] == df['class'][j]:
        pairs.append([df['text'][i], df['text'][j]])
        labels.append(0)

    # Negative pair
    else:
        while True:
            k = np.random.choice(df.index)
            if df['class'][k] != df['class'][i]:
                pairs.append([df['text'][i], df['text'][k]])
                labels.append(1)
                break

# Create a new dataframe with the pairs and labels
new_df = pd.DataFrame({'prompt1': [p[0] for p in pairs], 
                       'prompt2': [p[1] for p in pairs], 
                       'class': labels})

# Save the new dataframe to a CSV file
new_df

,prompt1,prompt2,class
0,news ncaas decision to cancel the big dance is...,news legendary chef michel roux dies aged 78 n...,0
1,news apple says its fine to wipe your iphone w...,news together we make a family of four n i hav...,0
2,news locked up and learning to write women pri...,news step into the world of americas most noto...,0
3,news opinion a story from inside the coronavi...,news they plan to march for gender equality in...,0
4,news roast chicken in a butter crust recipe n ...,news vaping whats a parent to do n the answer ...,0
...,...,...,...
10995,new packaging and plantbased fish how bumble b...,local news in and as long until it is that too...,1
10996,nikki haley an unprecedented step on human rig...,canada faces danger from china and russia inte...,0
10997,without god life is still meaningful opinion w...,how do you beat highflying liverpool watfords ...,1
10998,makoko nigerias floating slum goes digital mak...,is it allergies the flu or the coronavirus how...,0


LSTM

In [12]:
# Simple Neural Model

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

# Read in the dataset
df = new_df

# Split the dataset into samples with label 0 and 1
label_0_df = df[df['class'] == 0].sample(n=1000, random_state=42)
label_1_df = df[df['class'] == 1].sample(n=1000, random_state=42)

# Combine the two samples
sample_df = pd.concat([label_0_df, label_1_df])

# Convert the text prompts to numerical features using Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sample_df[['prompt1', 'prompt2']].apply(lambda x: ' '.join(x), axis=1))
X = tokenizer.texts_to_sequences(sample_df[['prompt1', 'prompt2']].apply(lambda x: ' '.join(x), axis=1))

# Pad the sequences to a fixed length
max_length = 300
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=max_length, padding='post', truncating='post')

# Add a third dimension to X
X = np.expand_dims(X, axis=2)

# Convert the labels to be in the range [0, 1]
y = sample_df['class'].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model architecture
model = Sequential([
    LSTM(64, input_shape=(max_length, 1)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on test data
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')


Epoch 1/10
50/50 [==============================] - 4s 29ms/step - loss: 0.7655 - accuracy: 0.5000 - val_loss: 0.6942 - val_accuracy: 0.5175
Epoch 2/10
50/50 [==============================] - 1s 12ms/step - loss: 0.7611 - accuracy: 0.4819 - val_loss: 0.6908 - val_accuracy: 0.5475
Epoch 3/10
50/50 [==============================] - 1s 11ms/step - loss: 0.7239 - accuracy: 0.5312 - val_loss: 0.6910 - val_accuracy: 0.5300
Epoch 4/10
50/50 [==============================] - 1s 11ms/step - loss: 0.7372 - accuracy: 0.4869 - val_loss: 0.6911 - val_accuracy: 0.4975
Epoch 5/10
50/50 [==============================] - 1s 12ms/step - loss: 0.7176 - accuracy: 0.5156 - val_loss: 0.6879 - val_accuracy: 0.5525
Epoch 6/10
50/50 [==============================] - 1s 11ms/step - loss: 0.7118 - accuracy: 0.5300 - val_loss: 0.6883 - val_accuracy: 0.5475
Epoch 7/10
50/50 [==============================] - 1s 13ms/step - loss: 0.7267 - accuracy: 0.4944 - val_loss: 0.6878 - val_accuracy: 0.5525
Epoch 8/10
50

BidirectionalLSTM

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding, SpatialDropout1D

# Read in the dataset
# df = pd.read_csv('p1_dataset.csv')
df = new_df

# # Split the dataset into samples with label 0 and 1
# label_0_df = df[df['class'] == 0].sample(n=10000, random_state=42)
# label_1_df = df[df['class'] == 1].sample(n=10000, random_state=42)

# Sample from each label, reducing the requested size if necessary
sample_size = 10000
if len(label_0_df) < sample_size:
    sample_size = len(label_0_df)
if len(label_1_df) < sample_size:
    sample_size = len(label_1_df)

label_0_df = label_0_df.sample(n=sample_size, random_state=42, replace=True)
label_1_df = label_1_df.sample(n=sample_size, random_state=42, replace=True)

# Combine the two samples
sample_df = pd.concat([label_0_df, label_1_df])

# Convert the text prompts to numerical features using Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sample_df[['prompt1', 'prompt2']].apply(lambda x: ' '.join(x), axis=1))
X = tokenizer.texts_to_sequences(sample_df[['prompt1', 'prompt2']].apply(lambda x: ' '.join(x), axis=1))

# Pad the sequences to a fixed length
max_length = 300
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=max_length, padding='post', truncating='post')

# Add a third dimension to X
X = np.expand_dims(X, axis=2)

# Convert the labels to be in the range [0, 1]
y = sample_df['class'].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model architecture
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_length),
    SpatialDropout1D(0.2),
    Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on test data
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')


Epoch 1/10
50/50 [==============================] - 72s 1s/step - loss: 0.6861 - accuracy: 0.5794 - val_loss: 0.6655 - val_accuracy: 0.5975
Epoch 2/10
50/50 [==============================] - 67s 1s/step - loss: 0.4987 - accuracy: 0.7850 - val_loss: 0.5688 - val_accuracy: 0.6900
Epoch 3/10
50/50 [==============================] - 67s 1s/step - loss: 0.1741 - accuracy: 0.9531 - val_loss: 0.6945 - val_accuracy: 0.7125
Epoch 4/10
50/50 [==============================] - 66s 1s/step - loss: 0.0838 - accuracy: 0.9775 - val_loss: 0.7819 - val_accuracy: 0.7225
Epoch 5/10
50/50 [==============================] - 65s 1s/step - loss: 0.0810 - accuracy: 0.9775 - val_loss: 0.8981 - val_accuracy: 0.6975
Epoch 6/10
50/50 [==============================] - 63s 1s/step - loss: 0.0580 - accuracy: 0.9812 - val_loss: 0.9787 - val_accuracy: 0.7025
Epoch 7/10
50/50 [==============================] - 63s 1s/step - loss: 0.0492 - accuracy: 0.9819 - val_loss: 0.8835 - val_accuracy: 0.7450
Epoch 8/10
50/50 [==

BI-LSTm with L2 Regularizer

In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding, SpatialDropout1D
from tensorflow.keras.regularizers import l2

# Read in the dataset
sample_df=new_df

# Convert the text prompts to numerical features using Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sample_df[['prompt1', 'prompt2']].apply(lambda x: ' '.join(x), axis=1))
X = tokenizer.texts_to_sequences(sample_df[['prompt1', 'prompt2']].apply(lambda x: ' '.join(x), axis=1))

# Pad the sequences to a fixed length
max_length = 300
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=max_length, padding='post', truncating='post')

# Add a third dimension to X
X = np.expand_dims(X, axis=2)

# Convert the labels to be in the range [0, 1]
y = sample_df['class'].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model architecture
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_length),
    SpatialDropout1D(0.2),
    Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)),
    Dropout(0.5),
    Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01))
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on test data
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')


Epoch 1/10
275/275 [==============================] - 365s 1s/step - loss: 0.6475 - accuracy: 0.6390 - val_loss: 0.6234 - val_accuracy: 0.6427
Epoch 2/10
275/275 [==============================] - 345s 1s/step - loss: 0.5399 - accuracy: 0.7323 - val_loss: 0.6581 - val_accuracy: 0.6123
Epoch 3/10
275/275 [==============================] - 336s 1s/step - loss: 0.4157 - accuracy: 0.8190 - val_loss: 0.7540 - val_accuracy: 0.6105
Epoch 4/10
275/275 [==============================] - 323s 1s/step - loss: 0.3390 - accuracy: 0.8608 - val_loss: 0.7864 - val_accuracy: 0.6182
Epoch 5/10
275/275 [==============================] - 325s 1s/step - loss: 0.2965 - accuracy: 0.8760 - val_loss: 0.9000 - val_accuracy: 0.6073
Epoch 6/10
275/275 [==============================] - 314s 1s/step - loss: 0.2694 - accuracy: 0.8799 - val_loss: 0.9213 - val_accuracy: 0.6068
Epoch 7/10
275/275 [==============================] - 323s 1s/step - loss: 0.2507 - accuracy: 0.8865 - val_loss: 0.8804 - val_accuracy: 0.6259

Logistic Regression

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Read in the dataset
# df = pd.read_csv('p1_dataset.csv')
df = new_df

# Sample from each label, reducing the requested size if necessary
sample_size = 10000
if len(label_0_df) < sample_size:
    sample_size = len(label_0_df)
if len(label_1_df) < sample_size:
    sample_size = len(label_1_df)

label_0_df = label_0_df.sample(n=sample_size, random_state=42, replace=True)
label_1_df = label_1_df.sample(n=sample_size, random_state=42, replace=True)


# Combine the two samples
sample_df = pd.concat([label_0_df, label_1_df])

# Convert the text prompts to a bag-of-words representation
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sample_df[['prompt1', 'prompt2']].apply(lambda x: ' '.join(x), axis=1))

# Convert the labels to be in the range [0, 1]
y = sample_df['class'].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model architecture
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Evaluate the model on test data
score = model.score(X_test, y_test)
print(f'Test accuracy: {score}')


Test accuracy: 0.895


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Random Forest

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Read in the dataset
# df = pd.read_csv('p1_dataset.csv')
df = new_df

# Sample from each label, reducing the requested size if necessary
sample_size = 10000
if len(label_0_df) < sample_size:
    sample_size = len(label_0_df)
if len(label_1_df) < sample_size:
    sample_size = len(label_1_df)

label_0_df = label_0_df.sample(n=sample_size, random_state=42, replace=True)
label_1_df = label_1_df.sample(n=sample_size, random_state=42, replace=True)


# Combine the two samples
sample_df = pd.concat([label_0_df, label_1_df])

# Convert the text prompts to a bag-of-words representation
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sample_df[['prompt1', 'prompt2']].apply(lambda x: ' '.join(x), axis=1))

# Convert the labels to be in the range [0, 1]
y = sample_df['class'].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model architecture
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model on test data
score = model.score(X_test, y_test)
print(f'Test accuracy: {score}')


Test accuracy: 0.93


In [ ]:
# Use the trained model to classify new prompt pairs
def classify_prompt_pair(prompt1, prompt2, model, vectorizer):
    prompt = vectorizer.transform([' '.join([prompt1, prompt2])])
    y_pred = model.predict(prompt)[0]
    if y_pred == 0:
        return 'generated by the same method'
    else:
        return 'generated by different methods'

In [ ]:
prompt1 = "she makes sure unwanted food gets to hungry americans cnn  the online tipping scheme employing the people to ask guests to pay would go a long way to help make everyday objects more accessible to the hungry american people according to a new study from the foodgift club at mercy college in louisville kentuckynconsumers surveyed by the organization food for those in need indicated that most the supporters recognized that a person has food in his or her pocket when they ask guests who pay a tip the only way possible according to the reportnfiftyseven percent of those surveyed 55 million people agreed that theres an urgent need for a communitywide initiative to assist millions of americans who do not have enough to eat in these hungry homesnthe survey was conducted in march by the food for those in need organization based at the university of louisvillenfood for those in need officials said in a statement that the organization plans to partner with the nonprofit organization to find a way to help poor americansnthe hungry citizens  the most in need of hunger assistance nationwide said evans follin who oversees the organizations marketing and distribution we know that americans are fed up and hunger is one of the top priorities for the federal government millions of americans are facing a minimum budget shortfall as a result of the current economic crisis with food insecurity mounting and the number of americans still unable to purchase basic goods such as small essentials such as milk eggs and produce risingnin the department of agricultures rural development and production program more than onethird of the population or an estimated 14 million people are food insecure or at least have insufficient enough to eat according to the reportnconsumers who talk about hunger around food and benefits themselves are more likely to identify the opportunity as food from a local deli a convenience store or the butcher shop according to the food for those in need reportnconsumers who recommend someone to donate food to the hungry through a friend or family member indicated they feel empowered to make the need a priority when ordering food those food donations usually include coupons or tokens they would like to pass along to others who receive them the report saidnhumanity may look to and accept a few givers as part of an effort to provide additional food to a large number of the hungry americans at our very own a communitys number one cause of hunger the study saidnother positive charitable behaviors for individuals and families need to be emphasized in this group and that is why the food for those in need initiative focuses on finding partnersn"
prompt2 = "how budapest became a fine dining force to be reckoned with and you were the one who made it happen because of which was your special role in that regard why for what reason  why noteopeod i have heard all about this movie but never actually seen any footage from its production so as far we are concerned there is no evidence whatsoever on our part regarding anything being done by any other person or entity at least none directly connected within hungary nor outside thereof such people would certainly need to know more concerning their own activities if they really wanted us involved either here inside hungarian territory elsewhere than where now exist some sorta network involving various individuals both locally along side national level even further out into the international arena perhaps including foreign governments like those currently in power over iran syria libya egypt iraq afghanistan pakistan yemen somalia sudan tanzania kenya uganda rwanda burundi zaire etc whatever these people might want them to do then again after realizing how much damage has been caused already through just doing nothing could possibly justify continuing without action against anyone else yet another example thus far only one though many others may well arise later upon the realizations shown me throughout my life however until next time letll keep things simple enough between ourselves rather than making too complicated an issue since most likely nobody will care anyway regardless whether something happens otherwise besides myself personally when eventually everything comes together before everyone can finally see themselves clearly hence also the point behind keeping matters simple while still remaining open up instead having someone try hard trying to make sure everybody knows exactly whose fault each individual situation truly lies under whom ever heshe believes should take responsibility therefore giving away very little information beyond simply stating oneself responsible plus pointing fingers towards whoever does wrong once somebody realized himself first hand quite often times becomes impossible due mostly owing mainly down to the fact every situation changes considerably depending entirely solely based off personal opinion versus objective truth despite knowing better yourself right around today maybe tomorrow sometime soon thereafter although neither one shall always be true nonetheless whenever situations change sometimes difficult decisions must inevitably come forth especially considering certain factors given above namely the following ones first thing foremost please remember beforehand everyone doesnt deserve happiness except yourselveseop how did anybody get involved during filming itself unless specifically asked earlier according back home somewhere near budapest apparently few people seem to understand precisely how anyone got involved anywhere close sufficient quantity nevertheless thanks per usual wherever possible toward getting started properly beginning tonight hopefully shortly afterwards yes sir thankyou indeed yeah sir thanks thanks mister yes thank you mr thank you miss sorry miss sorry miss oh sorry miss oh sorry miss o sorry miss o too late miss late miss d dear d dear"
classification = classify_prompt_pair(prompt1, prompt2, model, vectorizer)
print(f'{classification}.')